In [2]:
%load_ext autoreload

In [3]:
import sys
sys.path.append("..") #this is to add the above folder to the package directory
import os
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas import ExcelWriter

# from pandas.plotting import register_matplotlib_converters
import plotly.express as px
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

In [4]:
%matplotlib inline 

In [5]:
init_notebook_mode(connected=True) #this will connect plotly to the java script

In [6]:
cf.go_offline() #this will allow you to use cufflinks offline

# Hourly Data for the HPPs

# The outputs of this script are: 
1. The capacity factor of the river segments in the reference scenario. 
2. The residual capacity of the river segments in the reference scenario. 
3. The weekly distribution of the flow (compare to the respective month)

To be able to convert the montly flow data to weekly flow, we need to know the share of the flow in each week of the month. 

In this part we will use the hourly data for the previous years to generate the weekly shares.

Weekly share = (flow in week_x of month_y)/(total flow of month_y)

How to get the monthly and weekly flow from hourly data?

Monthly flow: groupby('month').mean()

Weekly flow: groupby(['month','week']).mean()


### Reading the file and seting the dataframe

In [7]:
folder_path = os.path.join('Data', 'hydrological_data')
hourly_data = os.path.join(folder_path, 'hourly data 2001-2010 for all river segments 20200930.xlsx')
df = pd.read_excel(hourly_data)

In [8]:
df2 =df.drop('Date', axis=1).copy()

In [9]:
# Replace negetive values with zero then with NaN. 
df2[df2<0]=0
df2= df2.replace({0:np.nan, 0:np.nan})

In [10]:
df2['Date']=df['Date']

In [11]:
df2.rename({'Orhid lake outtflow':'Orhid_CT',
           'Globocica lake side flow':'Globo_CT',
           'Globocica lake outflow':'Globo_HPP',
          'Debar/Shpilje lake side flow':'Shpil_CT',
           'Debar/Shpilje HPP':'Shpil_HPP',
           'Skavica side flow':'Skavi_CT',
           'Skavica outflow':'Skavi_HPP',
           'White Drini outflow':'WDrin_CT',
           'Fierza lake side flow':'Fierz_CT',
           'Fierza lake outflow':'Fjerz_HPP',
           'Koman lake side flow':'Koman_CT',
           'Koman lake outflow':'Koman_HPP',
           'Vau I Dejes lake side flow':'VauDe_CT',
           'Vau I Dejes lake outflow':'VauDe_HPP',
          }, axis=1, inplace=True)
df2.head()

,Orhid_CT,Globo_CT,Globo_HPP,Shpil_CT,Shpil_HPP,Skavi_CT,Skavi_HPP,WDrin_CT,Fierz_CT,Fjerz_HPP,Koman_CT,Koman_HPP,VauDe_CT,VauDe_HPP,Date
0,15.066,12.692,27.758,26.797,54.555,NaN,21.440,17.926,11.024,50.39,16.51,66.90,5.17,72.07,2001-11-01 00:00:00
1,14.975,12.735,27.710,26.704,54.414,NaN,21.412,17.917,11.061,50.39,16.65,67.04,5.00,72.04,2001-11-01 01:00:00
2,14.736,12.878,27.614,26.345,53.959,NaN,21.363,17.902,11.125,50.39,16.61,67.00,4.99,71.99,2001-11-01 02:00:00
3,14.471,13.047,27.518,26.232,53.750,NaN,21.324,17.888,11.178,50.39,16.58,66.97,4.96,71.93,2001-11-01 03:00:00
4,14.311,13.112,27.423,26.120,53.543,NaN,21.282,17.874,11.234,50.39,16.55,66.94,4.94,71.88,2001-11-01 04:00:00


In [12]:
df2['Date']=df2['Date']
df2['year'] = df2['Date'].dt.year 
df2['month'] = df2['Date'].dt.month
df2['week'] = df2['Date'].dt.week
df2.head()

,Orhid_CT,Globo_CT,Globo_HPP,Shpil_CT,Shpil_HPP,Skavi_CT,Skavi_HPP,WDrin_CT,Fierz_CT,Fjerz_HPP,Koman_CT,Koman_HPP,VauDe_CT,VauDe_HPP,Date,year,month,week
0,15.066,12.692,27.758,26.797,54.555,NaN,21.440,17.926,11.024,50.39,16.51,66.90,5.17,72.07,2001-11-01 00:00:00,2001,11,44
1,14.975,12.735,27.710,26.704,54.414,NaN,21.412,17.917,11.061,50.39,16.65,67.04,5.00,72.04,2001-11-01 01:00:00,2001,11,44
2,14.736,12.878,27.614,26.345,53.959,NaN,21.363,17.902,11.125,50.39,16.61,67.00,4.99,71.99,2001-11-01 02:00:00,2001,11,44
3,14.471,13.047,27.518,26.232,53.750,NaN,21.324,17.888,11.178,50.39,16.58,66.97,4.96,71.93,2001-11-01 03:00:00,2001,11,44
4,14.311,13.112,27.423,26.120,53.543,NaN,21.282,17.874,11.234,50.39,16.55,66.94,4.94,71.88,2001-11-01 04:00:00,2001,11,44


In [13]:
# To check the number of weeks in each month
wim = df2[['Date','year','month','week','Orhid_CT','Globo_CT','Shpil_CT','Skavi_CT','WDrin_CT','Fierz_CT','Koman_CT','VauDe_CT']]
#wim2 = wim[wim['year']==2005]
wim3 = wim.groupby(['week','month']).mean()
#pd.set_option("display.max_rows", None, "display.max_columns", None)
wim4 = wim.groupby(['month','week']).mean()
wim.loc[wim['week']==53,'year'].unique()

array([2004, 2005, 2009, 2010], dtype=int64)

## Sorting the dataframe by years, months and weeks 

In [14]:
dfCT=df2.filter(like='_CT').astype('float')
dfCT['year'] = df2['Date'].dt.year 
dfCT['month'] = df2['Date'].dt.month
dfCT['week'] = df2['Date'].dt.week
dfCT.head()

,Orhid_CT,Globo_CT,Shpil_CT,Skavi_CT,WDrin_CT,Fierz_CT,Koman_CT,VauDe_CT,year,month,week
0,15.066,12.692,26.797,NaN,17.926,11.024,16.51,5.17,2001,11,44
1,14.975,12.735,26.704,NaN,17.917,11.061,16.65,5.00,2001,11,44
2,14.736,12.878,26.345,NaN,17.902,11.125,16.61,4.99,2001,11,44
3,14.471,13.047,26.232,NaN,17.888,11.178,16.58,4.96,2001,11,44
4,14.311,13.112,26.120,NaN,17.874,11.234,16.55,4.94,2001,11,44


In [15]:
# Since we do not have data for all months in 2001 and 2010, we will delete them to reduce the errors in averaging
# Deleting week 53

#dfCT = dfCT[(dfCT.year !=2001) & (dfCT.year != 2010) & (dfCT.week != 53)]
dfCT = dfCT[(dfCT.week != 53)]

In [26]:
#weeklyflow=dfCT.groupby(['month','week'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()
weeklyflow=dfCT.groupby(['week'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()
weeklyflow.tail()

<ipython-input-26-52afe050703b>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,week,Orhid_CT,Globo_CT,Shpil_CT,Skavi_CT,WDrin_CT,Fierz_CT,Koman_CT,VauDe_CT
47,48,23.213448,5.585963,20.840876,49.133924,86.555896,100.007151,135.918120,35.246371
48,49,22.669348,5.108231,19.430239,59.814334,138.860688,90.534432,171.378807,30.696955
49,50,21.993138,4.981758,24.357498,42.209763,117.299662,106.652871,144.583639,38.550952
50,51,24.760794,3.824734,19.156515,35.641183,88.391080,91.064613,98.497612,28.100344
51,52,24.604341,5.903856,23.458664,36.908679,97.141212,81.725196,101.904541,27.334737


## Calculate the capacity factor for all the river segments and save files

In [27]:
# Generating the weekly capacity factor
#CF = weeklyflow/(weeklyflow.max().drop(['month','week']))
CF = weeklyflow/(weeklyflow.max().drop(['week']))
CF['week']=weeklyflow['week']
CF

,Fierz_CT,Globo_CT,Koman_CT,Orhid_CT,Shpil_CT,Skavi_CT,VauDe_CT,WDrin_CT,week
0,0.816387,0.955053,0.612366,0.735956,0.672018,0.674211,0.631493,0.747183,1
1,0.919688,1.000000,0.771650,0.855632,0.689003,0.686380,0.761226,0.827727,2
2,0.886416,0.666886,0.655045,0.868323,0.365340,0.419573,0.860121,0.555101,3
3,0.596549,0.696790,0.620425,0.943053,0.375136,0.496153,0.842948,0.613954,4
4,0.563040,0.973768,0.418728,0.911247,0.423679,0.511793,0.611168,0.601829,5
5,0.522634,0.891099,0.659686,0.856531,0.381444,0.451531,0.573630,0.591116,6
6,0.582686,0.915985,0.655002,0.806835,0.548913,0.586084,0.789778,0.731331,7
7,0.964687,0.883559,0.594637,0.760010,0.716208,0.801380,0.856782,0.925192,8
8,0.986885,0.962550,0.826373,0.815982,0.692829,0.860411,0.943009,0.799533,9
9,1.000000,0.936912,0.749568,0.869328,0.643544,0.883340,0.763461,1.000000,10


In [28]:
CF.columns

Index(['Fierz_CT', 'Globo_CT', 'Koman_CT', 'Orhid_CT', 'Shpil_CT', 'Skavi_CT',
       'VauDe_CT', 'WDrin_CT', 'week'],
      dtype='object')

In [29]:
# Updating the names to match the technologies in the model
CF.rename({'Fierz_CT':'ALCWTCT1DD',
           'Globo_CT':'MKCWTCT1BD',
           'Koman_CT':'ALCWTCT2DD',
           'Orhid_CT':'MKCWTLK0BD',
           'Shpil_CT':'MKCWTCT2BD',
           'Skavi_CT':'MKCWTCT3DD',
           'VauDe_CT':'ALCWTCT3DD',
           'WDrin_CT':'XKCWTRS1WD'}, axis=1, inplace=True)

In [119]:
# Saving the results for each HPP in a seperate sheet

output_folder = os.path.join('Processed_data', 'Capacity_factors', 'Reference')
os.makedirs(output_folder, exist_ok = True)

names = CF.columns.drop(['week'])

weeklyflow.to_excel(os.path.join(output_folder,'REF_WeeklyFlow.xlsx'),index=False)

for name in names:
    dfname = CF[['week',name]].copy()
    for i in range(20,56):
        dfname['20{}'.format(i)]=CF[name].round(3)
        dfname.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
        dfname.drop([name], axis=1).to_excel(os.path.join(output_folder,'REF_CF_{}_20201030.xlsx'.format(name)),index=False)

# Calculating the residual capacity of the river segment:

In [21]:
#Note that the we are computing the residual capacity based on the average historical flow (groupby...mean()) 
# To get the residual capacity in the wet year you can use (groupby...max()), similliarly (min) can be used to represent the dry year:

df3=dfCT.groupby(['month'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()
df4 = (df3.max()*(3600*8760/1000000)).round(3) #which represnts the max averaege flow
#river_residual_cap.round(3)
#river_residual_cap.reset_index(inplace=True)

<ipython-input-21-a7310bb8d80f>:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [40]:
river_residual_cap.rename({'Fierz_CT':'ALCWTCT1DD',
           'Globo_CT':'MKCWTCT1BD',
           'Koman_CT':'ALCWTCT2DD',
           'Orhid_CT':'MKCWTLK0BD',
           'Shpil_CT':'MKCWTCT2BD',
           'Skavi_CT':'MKCWTCT3DD',
           'VauDe_CT':'ALCWTCT3DD',
           'WDrin_CT':'XKCWTRS1WD'}, axis=1, inplace=True)

In [41]:
river_residual_cap

month          378.432000
MKCWTLK0BD    1069.835343
MKCWTCT1BD     206.218007
MKCWTCT2BD    1271.793717
MKCWTCT3DD    1983.621747
XKCWTRS1WD    3778.389146
ALCWTCT1DD    3251.485212
ALCWTCT2DD    4022.049519
ALCWTCT3DD    1049.460883
dtype: float64

In [33]:
# Note that for the river segments the Total Max Capacity and Total Max Capacity should also be set to equal the residual capacity

In [34]:
output_folder = os.path.join('Processed_data', 'Residual_Capacity','Reference')
os.makedirs(output_folder, exist_ok = True)

river_residual_cap.to_excel(os.path.join(output_folder,'REF_Residual_Capacity(MCM)_all_rivers.xlsx')) 


In [61]:
# To get the lower activity limit

df3=dfCT.groupby(['month'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()
df5 = df3.sum().round(3) #which represnts the total averaege flow in one year or the lower activity limit
df3

<ipython-input-61-dc87e847b819>:3: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,month,Orhid_CT,Globo_CT,Shpil_CT,Skavi_CT,WDrin_CT,Fierz_CT,Koman_CT,VauDe_CT
0,1,30.761338,6.476645,25.331434,44.935529,106.374626,98.793178,124.915284,31.476642
1,2,30.322241,6.539130,25.081538,43.784237,106.666573,82.027620,116.743132,27.298539
2,3,30.828849,5.917990,35.292953,62.900233,119.811934,103.103920,124.871466,33.278186
3,4,33.924256,4.062728,40.328314,61.218492,74.713880,85.776492,114.174760,26.004024
4,5,31.680140,3.600959,28.424937,35.051613,50.744447,63.142794,85.520149,17.110447
5,6,28.234691,3.097516,19.390949,18.875550,33.485938,38.955116,73.348519,14.562098
6,7,25.569927,2.005908,14.257319,9.102941,20.654515,24.371331,30.356671,7.257783
7,8,21.969103,2.353081,11.944623,7.511572,18.382744,18.528091,29.551437,8.204235
8,9,16.979421,2.029799,13.131173,15.476428,24.701216,14.048429,48.142332,15.762159
9,10,22.080608,5.132584,19.808679,39.049603,64.340481,38.953835,119.905373,30.935403


In [53]:
df5.rename({'Fierz_CT':'ALCWTCT1DD',
           'Globo_CT':'MKCWTCT1BD',
           'Koman_CT':'ALCWTCT2DD',
           'Orhid_CT':'MKCWTLK0BD',
           'Shpil_CT':'MKCWTCT2BD',
           'Skavi_CT':'MKCWTCT3DD',
           'VauDe_CT':'ALCWTCT3DD',
           'WDrin_CT':'XKCWTRS1WD'}, axis=1, inplace=True)
df5

month           78.000
MKCWTLK0BD     319.677
MKCWTCT1BD      52.150
MKCWTCT2BD     277.415
MKCWTCT3DD     426.988
XKCWTRS1WD     804.264
ALCWTCT1DD     749.826
ALCWTCT2DD    1107.836
ALCWTCT3DD     272.372
dtype: float64

In [58]:
df6 = df2.filter(like='_CT').astype('float')
df6['year'] = df2['Date'].dt.year 
df6['month'] = df2['Date'].dt.month
df6['week'] = df2['Date'].dt.week
df6.head()

,Orhid_CT,Globo_CT,Shpil_CT,Skavi_CT,WDrin_CT,Fierz_CT,Koman_CT,VauDe_CT,year,month,week
0,15.066,12.692,26.797,NaN,17.926,11.024,16.51,5.17,2001,11,44
1,14.975,12.735,26.704,NaN,17.917,11.061,16.65,5.00,2001,11,44
2,14.736,12.878,26.345,NaN,17.902,11.125,16.61,4.99,2001,11,44
3,14.471,13.047,26.232,NaN,17.888,11.178,16.58,4.96,2001,11,44
4,14.311,13.112,26.120,NaN,17.874,11.234,16.55,4.94,2001,11,44


In [60]:
df7=df6.groupby(['month'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()
df8 = df7.sum().round(3) #which represnts the total averaege flow in one year or the lower activity limit
df7

<ipython-input-60-a8b2c91fc7f0>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,month,Orhid_CT,Globo_CT,Shpil_CT,Skavi_CT,WDrin_CT,Fierz_CT,Koman_CT,VauDe_CT
0,1,30.752605,6.644723,26.263361,46.815568,108.295330,99.807132,127.316245,31.561442
1,2,30.322241,6.539130,25.081538,43.784237,106.666573,82.027620,116.743132,27.298539
2,3,30.828849,5.917990,35.292953,62.900233,119.811934,103.103920,124.871466,33.278186
3,4,33.924256,4.062728,40.328314,61.218492,74.713880,85.776492,114.174760,26.004024
4,5,31.680140,3.600959,28.424937,35.051613,50.744447,63.142794,85.520149,17.110447
5,6,28.234691,3.097516,19.390949,18.875550,33.485938,38.955116,73.348519,14.562098
6,7,25.569927,2.005908,14.257319,9.102941,20.654515,24.371331,30.356671,7.257783
7,8,21.969103,2.353081,11.944623,7.511572,18.382744,18.528091,29.551437,8.204235
8,9,16.979421,2.029799,13.131173,15.476428,24.701216,14.048429,48.142332,15.762159
9,10,22.080608,5.132584,19.808679,39.049603,64.340481,38.953835,119.905373,30.935403


In [52]:
output_folder = os.path.join('Processed_data', 'Residual_Capacity','Reference')
os.makedirs(output_folder, exist_ok = True)

df5.to_excel(os.path.join(output_folder,'REF_lowerActivityLimit_all_rivers.xlsx')) 

PermissionError: [WinError 31] A device attached to the system is not functioning: 'Processed_data'

# Weekly flow distribution

In [35]:
# Reading the file. You can skip this step if the dataframe is open

In [76]:
folder_path = os.path.join('Processed_data', 'Capacity_factors','Reference')
input_data = os.path.join(folder_path, 'REF_WeeklyFlow.xlsx')
weeklyflow = pd.read_excel(input_data)
weeklyflow = weeklyflow[(weeklyflow.week != 53)]

In [79]:
weeklyflow.rename({'Fierz_CT':'ALCWTCT1DD',
           'Globo_CT':'MKCWTCT1BD',
           'Koman_CT':'ALCWTCT2DD',
           'Orhid_CT':'MKCWTLK0BD',
           'Shpil_CT':'MKCWTCT2BD',
           'Skavi_CT':'MKCWTCT3DD',
           'VauDe_CT':'ALCWTCT3DD',
           'WDrin_CT':'XKCWTRS1WD'}, axis=1, inplace=True)
weeklyflow.columns

Index(['week', 'MKCWTLK0BD', 'MKCWTCT1BD', 'MKCWTCT2BD', 'MKCWTCT3DD',
       'XKCWTRS1WD', 'ALCWTCT1DD', 'ALCWTCT2DD', 'ALCWTCT3DD', 'month'],
      dtype='object')

In [77]:
folder_path = os.path.join('Data')
input_data = os.path.join(folder_path, 'months_and_weeks.xlsx')
monthsweeks = pd.read_excel(input_data)

In [78]:
weeklyflow_merged = pd.merge(weeklyflow, monthsweeks, on='week') 

In [80]:
weeklyflow_merged.tail()

,week,MKCWTLK0BD,MKCWTCT1BD,MKCWTCT2BD,MKCWTCT3DD,XKCWTRS1WD,ALCWTCT1DD,ALCWTCT2DD,ALCWTCT3DD,month
47,48,23.213448,5.585963,20.840876,49.133924,86.555896,100.007151,135.918120,35.246371,11
48,49,22.669348,5.108231,19.430239,59.814334,138.860688,90.534432,171.378807,30.696955,12
49,50,21.993138,4.981758,24.357498,42.209763,117.299662,106.652871,144.583639,38.550952,12
50,51,24.760794,3.824734,19.156515,35.641183,88.391080,91.064613,98.497612,28.100344,12
51,52,24.604341,5.903856,23.458664,36.908679,97.141212,81.725196,101.904541,27.334737,12


In [81]:
weeklyflow_melted = pd.melt(weeklyflow_merged, id_vars=['month','week'],
                     var_name='catchments', value_name='weeklyflow')

In [82]:
weeklyflow_melted.sort_values(['week', 'catchments'], inplace=True)

weeklyflow_melted

,month,week,catchments,weeklyflow
260,1,1,ALCWTCT1DD,101.297977
312,1,1,ALCWTCT2DD,115.977427
364,1,1,ALCWTCT3DD,25.140727
52,1,1,MKCWTCT1BD,7.070460
104,1,1,MKCWTCT2BD,31.905942
...,...,...,...,...
103,12,52,MKCWTCT1BD,5.903856
155,12,52,MKCWTCT2BD,23.458664
207,12,52,MKCWTCT3DD,36.908679
51,12,52,MKCWTLK0BD,24.604341


In [83]:
monthlyflow = dfCT.groupby(['month'])['Orhid_CT', 'Globo_CT', 'Shpil_CT', 'Skavi_CT', 'WDrin_CT', 'Fierz_CT','Koman_CT', 'VauDe_CT'].mean().reset_index()

#monthlyflow=weeklyflow.groupby(['month'])['MKCWTLK0BD', 'MKCWTCT1BD', 'MKCWTCT2BD', 'MKCWTCT3DD','XKCWTRS1WD', 'ALCWTCT1DD', 'ALCWTCT2DD', 'ALCWTCT3DD'].mean().reset_index()
monthlyflow.head()

<ipython-input-83-94a3398d3db5>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,month,Orhid_CT,Globo_CT,Shpil_CT,Skavi_CT,WDrin_CT,Fierz_CT,Koman_CT,VauDe_CT
0,1,30.761338,6.476645,25.331434,44.935529,106.374626,98.793178,124.915284,31.476642
1,2,30.322241,6.539130,25.081538,43.784237,106.666573,82.027620,116.743132,27.298539
2,3,30.828849,5.917990,35.292953,62.900233,119.811934,103.103920,124.871466,33.278186
3,4,33.924256,4.062728,40.328314,61.218492,74.713880,85.776492,114.174760,26.004024
4,5,31.680140,3.600959,28.424937,35.051613,50.744447,63.142794,85.520149,17.110447


In [96]:
monthlyflow.rename({'Fierz_CT':'ALCWTCT1DD',
           'Globo_CT':'MKCWTCT1BD',
           'Koman_CT':'ALCWTCT2DD',
           'Orhid_CT':'MKCWTLK0BD',
           'Shpil_CT':'MKCWTCT2BD',
           'Skavi_CT':'MKCWTCT3DD',
           'VauDe_CT':'ALCWTCT3DD',
           'WDrin_CT':'XKCWTRS1WD'}, axis=1, inplace=True)
monthlyflow.columns

Index(['month', 'MKCWTLK0BD', 'MKCWTCT1BD', 'MKCWTCT2BD', 'MKCWTCT3DD',
       'XKCWTRS1WD', 'ALCWTCT1DD', 'ALCWTCT2DD', 'ALCWTCT3DD'],
      dtype='object')

In [97]:
monthlyflow_merged = pd.merge(monthlyflow, monthsweeks, on='month') 

In [98]:
monthlyflow_merged

,month,MKCWTLK0BD,MKCWTCT1BD,MKCWTCT2BD,MKCWTCT3DD,XKCWTRS1WD,ALCWTCT1DD,ALCWTCT2DD,ALCWTCT3DD,week
0,1,30.761338,6.476645,25.331434,44.935529,106.374626,98.793178,124.915284,31.476642,1
1,1,30.761338,6.476645,25.331434,44.935529,106.374626,98.793178,124.915284,31.476642,2
2,1,30.761338,6.476645,25.331434,44.935529,106.374626,98.793178,124.915284,31.476642,3
3,1,30.761338,6.476645,25.331434,44.935529,106.374626,98.793178,124.915284,31.476642,4
4,1,30.761338,6.476645,25.331434,44.935529,106.374626,98.793178,124.915284,31.476642,5
5,2,30.322241,6.539130,25.081538,43.784237,106.666573,82.027620,116.743132,27.298539,6
6,2,30.322241,6.539130,25.081538,43.784237,106.666573,82.027620,116.743132,27.298539,7
7,2,30.322241,6.539130,25.081538,43.784237,106.666573,82.027620,116.743132,27.298539,8
8,2,30.322241,6.539130,25.081538,43.784237,106.666573,82.027620,116.743132,27.298539,9
9,3,30.828849,5.917990,35.292953,62.900233,119.811934,103.103920,124.871466,33.278186,10


In [99]:
monthlyflow_melted = pd.melt(monthlyflow_merged, id_vars=['month','week'],
                     var_name='catchments', value_name='monthlyflow')

In [104]:
monthlyflow_melted.sort_values(['month', 'catchments'], inplace=True)
monthlyflow_melted

,month,week,catchments,monthlyflow
260,1,1,ALCWTCT1DD,98.793178
261,1,2,ALCWTCT1DD,98.793178
262,1,3,ALCWTCT1DD,98.793178
263,1,4,ALCWTCT1DD,98.793178
264,1,5,ALCWTCT1DD,98.793178
...,...,...,...,...
51,12,52,MKCWTLK0BD,23.492288
256,12,49,XKCWTRS1WD,110.451803
257,12,50,XKCWTRS1WD,110.451803
258,12,51,XKCWTRS1WD,110.451803


In [109]:
dff = pd.merge(monthlyflow_melted, weeklyflow_melted, on=['week','catchments'])
dff.drop('month_y', axis=1, inplace=True)
dff.rename({'month_x':'month'}, axis=1, inplace=True)


,month,week,catchments,monthlyflow,weeklyflow
411,12,52,MKCWTLK0BD,23.492288,24.604341
412,12,49,XKCWTRS1WD,110.451803,138.860688
413,12,50,XKCWTRS1WD,110.451803,117.299662
414,12,51,XKCWTRS1WD,110.451803,88.391080
415,12,52,XKCWTRS1WD,110.451803,97.141212


In [117]:
dff.sort_values(['month','week','catchments'], inplace=True)

dff['weekpermonth']=dff['weeklyflow']/dff['monthlyflow']
dff.round(3)

,month,week,catchments,monthlyflow,weeklyflow,weekpermonth
0,1,1,ALCWTCT1DD,98.793,101.298,1.025
5,1,1,ALCWTCT2DD,124.915,115.977,0.928
10,1,1,ALCWTCT3DD,31.477,25.141,0.799
15,1,1,MKCWTCT1BD,6.477,7.070,1.092
20,1,1,MKCWTCT2BD,25.331,31.906,1.260
...,...,...,...,...,...,...
399,12,52,MKCWTCT1BD,5.097,5.904,1.158
403,12,52,MKCWTCT2BD,21.161,23.459,1.109
407,12,52,MKCWTCT3DD,44.365,36.909,0.832
411,12,52,MKCWTLK0BD,23.492,24.604,1.047


In [84]:

#dff.loc[(dff['month']==12)&(dff['catchment']=='Fierz_CT')]['weekpermonth'].sum()
# Note that the weekly disctrubution is not supposed to sum up to 1 as we are comparing the weekly average with the monthly average not the monthly sum.

In [118]:
#Writing results
output_folder = os.path.join('Processed_data', 'Weekly_distribution')
os.makedirs(output_folder, exist_ok = True)

file_path=os.path.join(output_folder, 'REF_WeeklyDistribution_AllCatchments_20201029.xlsx')
writer = pd.ExcelWriter(file_path, engine='xlsxwriter') 
dff.to_excel(writer,index=False)
    
writer.save()

In [137]:
# This is an alternative approach- Do no use it for now

rivers = globo.columns.drop(['Date', 'year', 'month', 'week'])

for river in rivers:
    globo_pvt = pd.pivot_table(globo,
                       values=river,
                       index=['month', 'week'],
                       columns=['year'],
                       aggfunc=np.mean)
    globo_pvt.drop([2001,2010],axis=1, inplace=True)
    #globo_pvt.drop([53], inplace=True)
    globo_pvt.reset_index(inplace=True)
    globo_pvt.fillna(0, inplace=True) #relace all NaN value with zero
    
    globo2= globo_pvt[(globo_pvt!=0).all(axis=1)] #To delete all the rows with zero values
    globo2['weekly_mean'] = globo2.iloc[:,[2,3,4,5,6,7,8,9]].mean(axis=1)
    
    monthly_flow = globo2.groupby(['month']).mean()
    monthly_flow.reset_index(inplace=True)

    monthly_flow['monthly_mean'] = monthly_flow.iloc[:,[2,3,4,5,6,7,8,9]].mean(axis=1)

    df_monthly = monthly_flow[['month','monthly_mean']]
    df_merge = pd.merge(globo2, df_monthly, on='month')
    df_merge['weekly_share'] = df_merge['weekly_mean']/df_merge['monthly_mean']
    dff = df_merge[['month','week','monthly_mean','weekly_mean','weekly_share' ]].copy().round(3)
    #Writing results
    output_folder = os.path.join('Processed_data', 'Weekly_distribution')
    os.makedirs(output_folder, exist_ok = True)

    file_path=os.path.join(output_folder, '{}'.format(river)+'_weekly_share.xlsx')
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter') 
    dff.to_excel(writer , index=False)
    
    writer.save()

In [ ]:
#end